# [실습] CNN활용_전이학습_InceptionV3_transfer_learning

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np


#### 데이터 수집

In [2]:
# MNIST 데이터셋을 로드하고, 정규화한다
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train[:10000] / 255.0, x_test[:2000] / 255.0
#x_train, x_test = x_train[:3] / 255.0, x_test[:2000] / 255.0
## 훈련데이터는 10000개, 검증 데이터는 2000개만 사용
y_train, y_test = y_train[:10000], y_test[:2000]

print(x_train.shape)

(10000, 28, 28)


#### 사전 학습된 Inceptionv3 모델 살펴보기

In [3]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras import layers
from tensorflow.keras.optimizers import RMSprop

In [4]:
model = InceptionV3()
model.summary()

96112376/96112376 [==============================] - 13s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)          

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True, show_layer_names=True, to_file='InceptionV3.png')

In [11]:
from skimage.transform import resize

x_train_preprocess = np.zeros((x_train.shape[0], 75, 75, 3), 
                            dtype=np.float32)
print(x_train_preprocess.shape)

(10000, 75, 75, 3)


In [12]:

for i, img in enumerate(x_train):
    img_resize = resize(img, (75, 75), anti_aliasing=True)
    x_train_preprocess[i] = np.dstack([img_resize, img_resize, img_resize])

x_test_preprocess = np.zeros((x_test.shape[0], 75, 75, 3), 
                            dtype=np.float32)

print(x_test_preprocess.shape)

(2000, 75, 75, 3)


In [13]:
# 검증 데이터 각각의 크기를 75x75로 변형 (인셉션모델 최소 입력 크기)
for i, img in enumerate(x_test):
    img_resize = resize(img, (75, 75), anti_aliasing=True)
    x_test_preprocess[i] = np.dstack([img_resize, img_resize, img_resize])

In [14]:
model = InceptionV3(input_shape=(75, 75, 3), include_top=False, )

87910968/87910968 [==============================] - 3s 0us/step


In [17]:
pretrained_model = model

In [15]:
for layer in model.layers:
    layer.trainable = False

In [16]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 75, 75, 3)]  0           []                               
                                                                                                  
 conv2d_94 (Conv2D)             (None, 37, 37, 32)   864         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_94 (BatchN  (None, 37, 37, 32)  96          ['conv2d_94[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_94 (Activation)     (None, 37, 37, 32)   0           ['batch_normalization_

In [18]:
last_layer = pretrained_model.get_layer('mixed7')
last_output = last_layer.output

x1 = layers.Flatten()(last_output)
x2 = layers.Dense(64, activation='relu')(x1)
x3 = layers.Dense(10, activation='softmax')(x2)

model = Model(pretrained_model.input, x3) # (시작, 끝)

model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics='acc')

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 75, 75, 3)]  0           []                               
                                                                                                  
 conv2d_94 (Conv2D)             (None, 37, 37, 32)   864         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_94 (BatchN  (None, 37, 37, 32)  96          ['conv2d_94[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_94 (Activation)     (None, 37, 37, 32)   0           ['batch_normalization_94[0][0

In [19]:
history = model.fit(x_train_preprocess, y_train, epochs=20, validation_data=(x_test_preprocess, y_test))

Epoch 1/20
313/313 [==============================] - 34s 96ms/step - loss: 0.2566 - acc: 0.9258 - val_loss: 0.1014 - val_acc: 0.9705
Epoch 2/20
313/313 [==============================] - 30s 96ms/step - loss: 0.0698 - acc: 0.9796 - val_loss: 0.0995 - val_acc: 0.9695
Epoch 3/20
313/313 [==============================] - 30s 97ms/step - loss: 0.0369 - acc: 0.9896 - val_loss: 0.0868 - val_acc: 0.9720
Epoch 4/20
313/313 [==============================] - 30s 97ms/step - loss: 0.0329 - acc: 0.9897 - val_loss: 0.0785 - val_acc: 0.9740
Epoch 5/20
313/313 [==============================] - 31s 97ms/step - loss: 0.0200 - acc: 0.9938 - val_loss: 0.0983 - val_acc: 0.9730
Epoch 6/20
313/313 [==============================] - 30s 97ms/step - loss: 0.0190 - acc: 0.9940 - val_loss: 0.0946 - val_acc: 0.9715
Epoch 7/20
313/313 [==============================] - 30s 96ms/step - loss: 0.0111 - acc: 0.9970 - val_loss: 0.0948 - val_acc: 0.9715
Epoch 8/20
313/313 [==============================] - 31s 99ms

In [20]:
# 학습을 마치면 모델을 저장하는 습관을 들이기!!!!!!!!
model.save('252.1_InceptionV3.h5')

In [22]:
import pandas as pd

pd.DataFrame(history.history)

,loss,acc,val_loss,val_acc
0,0.256552,0.9258,0.101403,0.9705
1,0.069763,0.9796,0.099508,0.9695
2,0.036927,0.9896,0.086772,0.9720
3,0.032907,0.9897,0.078516,0.9740
4,0.019954,0.9938,0.098293,0.9730
5,0.019006,0.9940,0.094624,0.9715
6,0.011109,0.9970,0.094826,0.9715
7,0.012819,0.9958,0.075593,0.9750
8,0.006653,0.9986,0.100143,0.9730
9,0.027214,0.9910,0.091659,0.9775


In [ ]:
# 손실함수, 정확도 그래프 그리기


### 구글 Inception V3를 활용한 손글씨 MNIST 분류(ver1.0)

## 구글 Inception V3를 활용한 손글씨 MNIST 분류(ver2.0)